In [ ]:
from __future__ import annotations

from dataclasses import dataclass, field
from typing import Dict, List, Optional
from enum import Enum
import matplotlib.pyplot as plt
import random
import numpy as np

In [4]:
class Sector(Enum):
    ENERGY = "energy"
    CONSTRUCTION = "construction"
    MANUFACTURING = "manufacturing"
    LOGISTICS = "logistics"
    RETAIL = "retail"
    TELECOM = "telecom"
    TRANSPORT = "transport"
    FINANCE = "finance"
    METALS = "metals"


In [5]:
class MarketStructure(Enum):
    MONOPOLY = "monopoly"
    COMPETITIVE = "competitive"
    COURNOT = "cournot"
    STACKELBERG_LEADER = "stackelberg_leader"
    STACKELBERG_FOLLOWER = "stackelberg_follower"


In [6]:
@dataclass(frozen=True)
class Shock:
    name: str
    description: str
    demand_effects: Dict[Sector, float]
    cost_effects: Dict[Sector, float]
    metal_effects: Dict[str, float]
    interest_rate_delta: float = 0.0


In [7]:
SHOCKS: List[Shock] = [
    Shock(
        "Эмбарго на нефть",
        "Запрет экспорта нефти",
        demand_effects={Sector.ENERGY: 0.6},
        cost_effects={Sector.TRANSPORT: 1.25},
        metal_effects={"gold": 1.2}
    ),
    Shock(
        "Ипотечные субсидии",
        "Поддержка строительства",
        demand_effects={Sector.CONSTRUCTION: 1.45},
        cost_effects={},
        metal_effects={}
    ),
    Shock(
        "ИИ-прорыв",
        "Рост производительности",
        demand_effects={Sector.TELECOM: 1.3, Sector.FINANCE: 1.2},
        cost_effects={Sector.MANUFACTURING: 0.85},
        metal_effects={"gold": 1.1}
    ),
    Shock(
        "Квантовая телепортация товаров",
        "Логистика исчезает",
        demand_effects={Sector.LOGISTICS: 0.1},
        cost_effects={Sector.LOGISTICS: 0.2},
        metal_effects={}
    ),
    Shock(
        "Углеродный налог",
        "Рост издержек промышленности",
        demand_effects={},
        cost_effects={Sector.ENERGY: 1.35, Sector.MANUFACTURING: 1.25},
        metal_effects={}
    ),
]


In [8]:
@dataclass
class EconomicEnvironment:
    active_shocks: List[Shock]
    base_interest_rate: float = 0.08
    round_num: int = 1

    def demand_multiplier(self, sector: Sector) -> float:
        m = 1.0
        for shock in self.active_shocks:
            m *= shock.demand_effects.get(sector, 1.0)
        return m

    def cost_multiplier(self, sector: Sector) -> float:
        m = 1.0
        for shock in self.active_shocks:
            m *= shock.cost_effects.get(sector, 1.0)
        return m

    def interest_rate(self) -> float:
        r = self.base_interest_rate
        for shock in self.active_shocks:
            r += shock.interest_rate_delta
        return max(r, 0.0)


In [9]:
@dataclass
class LinearDemand:
    a_base: float
    b: float
    sector: Sector

    def effective_a(self, env: EconomicEnvironment) -> float:
        return self.a_base * env.demand_multiplier(self.sector)

    def price(self, Q: float, env: EconomicEnvironment) -> float:
        return max(0.0, self.effective_a(env) - self.b * Q)

    def elasticity(self, Q: float, env: EconomicEnvironment) -> float:
        P = self.price(Q, env)
        if P == 0:
            return 0.0
        return -self.b * Q / P




In [10]:
@dataclass
class BankAccount:
    deposits: float = 0.0
    loans: float = 0.0

    def update(self, interest_rate: float) -> None:
        self.deposits *= (1 + interest_rate * 0.6)
        self.loans *= (1 + interest_rate)


In [11]:
@dataclass
class Player:
    name: str
    capital: float = field(default_factory=lambda: random.uniform(1000, 2000))
    bank: BankAccount = field(default_factory=BankAccount)
    owned_companies: List[str] = field(default_factory=list)

    def net_worth(self) -> float:
        return self.capital + self.bank.deposits - self.bank.loans


In [12]:
@dataclass
class CostFunction:
    fixed: float
    marginal_base: float
    sector: Sector

    def marginal_cost(self, env: EconomicEnvironment) -> float:
        return self.marginal_base * env.cost_multiplier(self.sector)

    def total(self, Q: float, env: EconomicEnvironment) -> float:
        return self.fixed + self.marginal_cost(env) * Q

    def average_cost(self, Q: float, env: EconomicEnvironment) -> float:
        if Q == 0:
            return float('inf')
        return self.total(Q, env) / Q


In [13]:
@dataclass
class Company:
    name: str
    sector: Sector
    demand: LinearDemand
    costs: CostFunction
    structure: MarketStructure
    competitors: int = 0
    owned_by_player: bool = False
    price: float = 1000.0
    current_production: float = 0.0

    def profit(self, Q: float, env: EconomicEnvironment) -> float:
        P = self.demand.price(Q, env)
        return P * Q - self.costs.total(Q, env)

    def buy(self, player: Player) -> bool:
        """Игрок пытается купить компанию"""
        if self.owned_by_player:
            print(f"Компания {self.name} уже куплена игроком.")
            return False
        if player.capital < self.price:
            print(f"Недостаточно капитала для покупки {self.name}. Требуется {self.price}, у вас {player.capital}.")
            return False
        player.capital -= self.price
        self.owned_by_player = True
        print(f"Компания {self.name} успешно куплена игроком. Остаток капитала: {player.capital}")
        return True

    def sell(self, player: Player) -> bool:
        """Игрок пытается продать компанию"""
        if not self.owned_by_player:
            print(f"Компания {self.name} не принадлежит игроку.")
            return False
        player.capital += self.price
        self.owned_by_player = False
        print(f"Компания {self.name} успешно продана. Капитал игрока: {player.capital}")
        return True

In [14]:
class Optimizer:
    """
    Теоретический решатель задач максимизации прибыли.
    Используется только для проверки решений игрока.
    """

    @staticmethod
    def _a(company: Company, env: EconomicEnvironment) -> float:
        return company.demand.effective_a(env)

    @staticmethod
    def _b(company: Company) -> float:
        return company.demand.b

    @staticmethod
    def _mc(company: Company, env: EconomicEnvironment) -> float:
        return company.costs.marginal_cost(env)

    # --- Монополия ---
    @staticmethod
    def monopoly_Q(company: Company, env: EconomicEnvironment) -> float:
        a, b, mc = (
            Optimizer._a(company, env),
            Optimizer._b(company),
            Optimizer._mc(company, env),
        )
        return max(0.0, (a - mc) / (2 * b))

    # --- Курно ---
    @staticmethod
    def cournot_Q(company: Company, env: EconomicEnvironment) -> float:
        a, b, mc = (
            Optimizer._a(company, env),
            Optimizer._b(company),
            Optimizer._mc(company, env),
        )
        n = company.competitors + 1
        return max(0.0, (a - mc) / (b * (n + 1)))

    # --- Штакельберг: ведомый ---
    @staticmethod
    def stackelberg_follower_Q(
        company: Company, env: EconomicEnvironment, leader_Q: float
    ) -> float:
        a, b, mc = (
            Optimizer._a(company, env),
            Optimizer._b(company),
            Optimizer._mc(company, env),
        )
        return max(0.0, (a - mc - b * leader_Q) / (2 * b))

    # --- Штакельберг: лидер ---
    @staticmethod
    def stackelberg_leader_Q(company: Company, env: EconomicEnvironment) -> float:
        return Optimizer.monopoly_Q(company, env)

    @staticmethod
    def optimal_Q(
        company: Company,
        env: EconomicEnvironment,
        leader_Q: Optional[float] = None
    ) -> float:

        if company.structure == MarketStructure.MONOPOLY:
            return Optimizer.monopoly_Q(company, env)

        if company.structure == MarketStructure.COURNOT:
            return Optimizer.cournot_Q(company, env)

        if company.structure == MarketStructure.STACKELBERG_LEADER:
            return Optimizer.stackelberg_leader_Q(company, env)

        if company.structure == MarketStructure.STACKELBERG_FOLLOWER:
            if leader_Q is None:
                raise ValueError("Нужен выпуск лидера")
            return Optimizer.stackelberg_follower_Q(company, env, leader_Q)

        # Конкурентный рынок: P = MC
        a, b, mc = (
            Optimizer._a(company, env),
            Optimizer._b(company),
            Optimizer._mc(company, env),
        )
        return max(0.0, (a - mc) / b)


In [15]:
def evaluate_decision(
    company: Company,
    Q_player: float,
    env: EconomicEnvironment,
    leader_Q: Optional[float] = None,
    tolerance: float = 0.05
) -> Dict[str, float | str]:

    # Определяем оптимальный объем производства
    Q_star = Optimizer.optimal_Q(company, env, leader_Q)

    # Рассчитываем отклонение
    deviation = abs(Q_player - Q_star) / max(Q_star, 1e-6)

    # Возвращаем только статус: "Верно" или "Неверно"
    return {
        "Q_player": Q_player,
        "Q_optimal": Q_star,
        "status": "Верно" if deviation <= tolerance else "Неверно",
    }

In [16]:
class Analytics:
    @staticmethod
    def plot_market(company: Company, Q_player: float, env: EconomicEnvironment) -> None:
        a_effective = company.demand.effective_a(env)
        b = company.demand.b
        mc = company.costs.marginal_cost(env)

        # Максимальный объем (когда P=0)
        Q_max = int(a_effective / b) + 10

        Q_range = np.linspace(0, Q_max, 100)

        # Спрос и предельная выручка
        demand = [company.demand.price(q, env) for q in Q_range]
        MR = [a_effective - 2 * b * q for q in Q_range]

        # Постоянные предельные издержки
        MC = [mc] * len(Q_range)

        # Средние издержки (убывающие из-за постоянных издержек)
        AC = [company.costs.average_cost(q, env) for q in Q_range]

        # Средние переменные издержки = MC (постоянны)
        AVC = [mc] * len(Q_range)

        plt.figure(figsize=(10, 6))

        # Основные кривые
        plt.plot(Q_range, demand, 'b-', label='Спрос (P)', linewidth=2)
        plt.plot(Q_range, MR, 'b--', label='Предельная выручка (MR)', linewidth=1.5, alpha=0.7)
        plt.plot(Q_range, MC, 'r-', label='Предельные издержки (MC)', linewidth=2)
        plt.plot(Q_range, AC, 'g-', label='Средние издержки (AC)', linewidth=2)
        plt.plot(Q_range, AVC, 'g--', label='Средние переменные издержки (AVC)', linewidth=1.5, alpha=0.7)

        # Оптимальная точка
        Q_opt = Optimizer.optimal_Q(company, env)
        P_opt = company.demand.price(Q_opt, env)

        plt.scatter([Q_opt], [P_opt], color='green', s=100, zorder=5,
                   label=f'Оптимум (Q={Q_opt:.1f}, P={P_opt:.1f})')

        # Решение игрока
        P_player = company.demand.price(Q_player, env)
        plt.scatter([Q_player], [P_player], color='red', s=100, zorder=5,
                   label=f'Ваше решение (Q={Q_player:.1f}, P={P_player:.1f})')

        # Заполнение областей прибыли/убытков
        if company.structure == MarketStructure.MONOPOLY:
            # Для монополии показываем прибыль
            profit_area_Q = np.linspace(0, Q_opt, 50)
            profit_area_P = [company.demand.price(q, env) for q in profit_area_Q]
            profit_area_MC = [mc] * len(profit_area_Q)

            plt.fill_between(profit_area_Q, profit_area_MC, profit_area_P,
                            alpha=0.2, color='green', label='Экономическая прибыль')

        plt.title(f'Рынок: {company.name}\nСтруктура: {company.structure.value}', fontsize=14)
        plt.xlabel("Объем производства (Q)", fontsize=12)
        plt.ylabel("Цена/Издержки", fontsize=12)
        plt.legend(loc='best')
        plt.grid(True, alpha=0.3)
        plt.tight_layout()
        plt.show()

In [17]:
companies = [
    Company(
        "ВШЭнефть",
        Sector.ENERGY,
        LinearDemand(900, 2, Sector.ENERGY),
        CostFunction(200, 50, Sector.ENERGY),
        MarketStructure.MONOPOLY
    ),
    Company(
        "ВШЭмагазин",
        Sector.RETAIL,
        LinearDemand(700, 2.2, Sector.RETAIL),
        CostFunction(100, 40, Sector.RETAIL),
        MarketStructure.STACKELBERG_FOLLOWER
    ),
    Company(
        "ВШЭлайна",
        Sector.TELECOM,
        LinearDemand(650, 2, Sector.TELECOM),
        CostFunction(110, 42, Sector.TELECOM),
        MarketStructure.COURNOT,
        competitors=1
    ),
    Company(
        "ВШЭстрой",
        Sector.TELECOM,
        LinearDemand(700, 4, Sector.CONSTRUCTION),
        CostFunction(400, 29, Sector.CONSTRUCTION),
        MarketStructure.COMPETITIVE,
    ),
]


**Тестируем приложение с покупкой**

In [18]:
class ConsoleGame:
    def __init__(self):
        self.player = Player("Игрок")  # Игрок, который будет участвовать в игре
        self.companies = self.create_companies()  # Список компаний
        self.env = EconomicEnvironment(active_shocks=[], round_num=1)  # Экономическая среда

    def create_companies(self) -> List[Company]:
        """Создание компаний с линейными издержками"""
        return [
            Company(
                "ВШЭнефть",
                Sector.ENERGY,
                LinearDemand(900, 2, Sector.ENERGY),
                CostFunction(200, 50, Sector.ENERGY),
                MarketStructure.MONOPOLY,
                price=1500.0
            ),
            Company(
                "ВШЭмагазин",
                Sector.RETAIL,
                LinearDemand(700, 2.2, Sector.RETAIL),
                CostFunction(100, 40, Sector.RETAIL),
                MarketStructure.STACKELBERG_FOLLOWER,
                price=1200.0
            ),
            Company(
                "ВШЭлайна",
                Sector.TELECOM,
                LinearDemand(650, 2, Sector.TELECOM),
                CostFunction(110, 42, Sector.TELECOM),
                MarketStructure.COURNOT,
                competitors=1,
                price=1300.0
            ),
            Company(
                "ВШЭстрой",
                Sector.CONSTRUCTION,
                LinearDemand(700, 4, Sector.CONSTRUCTION),
                CostFunction(400, 29, Sector.CONSTRUCTION),
                MarketStructure.COMPETITIVE,
                price=1100.0
            )
        ]

    def display_status(self):
        """Показать статус игры"""
        print("\n" + "="*50)
        print(f"РАУНД {self.env.round_num}")
        print("="*50)
        print(f"Игрок: {self.player.name}")
        print(f"Капитал: {self.player.capital:.2f} млн")
        print(f"Чистая стоимость: {self.player.net_worth():.2f} млн")
        print(f"Купленные компании: {', '.join(self.player.owned_companies) if self.player.owned_companies else 'Нет'}")

        if self.env.active_shocks:
            print(f"\nАктивные шоки:")
            for shock in self.env.active_shocks:
                print(f"  • {shock.name}: {shock.description}")
        else:
            print("\nАктивных шоков нет")

        print(f"Процентная ставка: {self.env.interest_rate():.1%}")

    def display_companies(self):
        """Показать доступные компании"""
        print("\n" + "-"*50)
        print("ДОСТУПНЫЕ КОМПАНИИ:")
        print("-"*50)
        for idx, company in enumerate(self.companies, 1):
            status = "✓ Ваша" if company.owned_by_player else "Свободна"
            print(f"{idx}. {company.name}")
            print(f"   Сектор: {company.sector}")
            print(f"   Структура рынка: {company.structure}")
            print(f"   Конкурентов: {company.competitors}")
            print(f"   Цена: {company.price:.2f} млн")
            print(f"   Статус: {status}")
            print()

    def apply_random_shock(self):
        """Применить случайный шок"""
        if random.random() < 0.3:  # 30% вероятность шока
            shock = random.choice(SHOCKS)
            self.env.active_shocks.append(shock)
            print(f"\n⚠️  ПРОИЗОШЕЛ ШОК: {shock.name}")
            print(f"   {shock.description}")

            # Показать эффекты шока
            for sector, effect in shock.demand_effects.items():
                if effect != 1.0:
                    change = "↑" if effect > 1.0 else "↓"
                    print(f"   • Спрос в секторе {sector}: {change} на {abs(effect-1.0)*100:.0f}%")

            for sector, effect in shock.cost_effects.items():
                if effect != 1.0:
                    change = "↑" if effect > 1.0 else "↓"
                    print(f"   • Издержки в секторе {sector}: {change} на {abs(effect-1.0)*100:.0f}%")

    def buy_company(self, idx: int):
        """Купить компанию"""
        if idx < 0 or idx >= len(self.companies):
            print("Неверный номер компании")
            return False

        company = self.companies[idx]
        return company.buy(self.player)

    def set_price_and_produce(self, idx: int):
        """Установить цену и объем производства"""
        company = self.companies[idx]

        if not company.owned_by_player:
            print("Сначала нужно купить эту компанию!")
            return

        print(f"\n{company.name}:")

        try:
            # Игрок вводит объем производства
            while True:
                try:
                    Q_player = float(input(f"Введите объем производства для {company.name} (0-{company.demand.a_base/company.demand.b:.0f}): "))
                    if Q_player < 0:
                        print("Объем не может быть отрицательным!")
                        continue
                    break
                except ValueError:
                    print("Пожалуйста, введите число")

            # Автоматически рассчитываем цену по кривой спроса
            P_player = company.demand.price(Q_player, self.env)
            company.current_production = Q_player
            company.price = P_player

            print(f"\nПри объеме Q = {Q_player:.1f}:")
            print(f"Цена будет: P = {P_player:.1f}")

            # Оценка решения
            print("\nОЦЕНКА ВАШЕГО РЕШЕНИЯ")

            evaluation = self.evaluate_decision(company, Q_player)

            # Выводим только статус "Верно" или "Неверно"
            print(f"Статус: {evaluation['status']}")

        except KeyboardInterrupt:
            print("\nОтменено")
        except Exception as e:
            print(f"Ошибка: {e}")

    def evaluate_decision(self, company: Company, Q_player: float) -> Dict:
        """Оценка решения игрока"""
        # Определение оптимального объема
        optimal_Q = company.demand.a_base / company.demand.b  # Оптимальный объем
        optimal_profit = company.profit(optimal_Q, self.env)  # Максимальная прибыль
        player_profit = company.profit(Q_player, self.env)  # Прибыль игрока

        status = "Верно" if abs(Q_player - optimal_Q) / max(optimal_Q, 1e-6) <= 0.05 else "Неверно"

        return {
            "Q_player": Q_player,
            "Q_optimal": optimal_Q,
            "player_profit": player_profit,
            "optimal_profit": optimal_profit,
            "status": status
        }

    def next_round(self):
        """Переход к следующему раунду"""
        self.env.round_num += 1

        # Очистка шоков (они действуют только один раунд)
        self.env.active_shocks = []

        # Обновление банковского счета
        self.player.bank.update(self.env.interest_rate())

        # Компании приносят прибыль
        for company in self.companies:
            if company.owned_by_player and company.current_production > 0:
                profit = company.profit(company.current_production, self.env)
                self.player.capital += profit
                print(f"Компания {company.name} принесла прибыль: {profit:.1f}")

        # Применяем новый случайный шок
        self.apply_random_shock()

    def play_turn(self):
        """Один ход игрока"""
        self.display_status()
        self.display_companies()

        print("\n" + "="*50)
        print("МЕНЮ ДЕЙСТВИЙ:")
        print("1. Купить компанию")
        print("2. Установить объем производства (цена определится автоматически)")
        print("3. Продать компанию")
        print("4. Перейти к следующему раунду")
        print("5. Завершить игру")

        try:
            action = int(input("\nВыберите действие: "))
        except ValueError:
            print("Пожалуйста, выберите действие числом.")
            return

        if action == 1:
            idx = int(input("Введите номер компании для покупки: ")) - 1
            self.buy_company(idx)
        elif action == 2:
            idx = int(input("Введите номер компании для производства: ")) - 1
            self.set_price_and_produce(idx)
        elif action == 3:
            idx = int(input("Введите номер компании для продажи: ")) - 1
            self.companies[idx].sell(self.player)
        elif action == 4:
            self.next_round()
        elif action == 5:
            print("Игра завершена.")
            exit()
        else:
            print("Неверный выбор!")


In [ ]:
def main():
    # Создание объекта игры
    game = ConsoleGame()

    # Цикл игры
    while True:
        game.play_turn()  # Игрок выполняет ход

if __name__ == "__main__":
    main()



РАУНД 1
Игрок: Игрок
Капитал: 1634.27 млн
Чистая стоимость: 1634.27 млн
Купленные компании: Нет

Активных шоков нет
Процентная ставка: 8.0%

--------------------------------------------------
ДОСТУПНЫЕ КОМПАНИИ:
--------------------------------------------------
1. ВШЭнефть
   Сектор: Sector.ENERGY
   Структура рынка: MarketStructure.MONOPOLY
   Конкурентов: 0
   Цена: 1500.00 млн
   Статус: Свободна

2. ВШЭмагазин
   Сектор: Sector.RETAIL
   Структура рынка: MarketStructure.STACKELBERG_FOLLOWER
   Конкурентов: 0
   Цена: 1200.00 млн
   Статус: Свободна

3. ВШЭлайна
   Сектор: Sector.TELECOM
   Структура рынка: MarketStructure.COURNOT
   Конкурентов: 1
   Цена: 1300.00 млн
   Статус: Свободна

4. ВШЭстрой
   Сектор: Sector.CONSTRUCTION
   Структура рынка: MarketStructure.COMPETITIVE
   Конкурентов: 0
   Цена: 1100.00 млн
   Статус: Свободна


МЕНЮ ДЕЙСТВИЙ:
1. Купить компанию
2. Установить объем производства (цена определится автоматически)
3. Продать компанию
4. Перейти к следующему 